# EDITH: ECG biometric aided by deep learning for reliable individual authentication

# PREPROCESSING

In [1]:
import numpy as np
from scipy.signal import savgol_filter
from pykalman import KalmanFilter

# Load ECG data from CSV file
ecg = np.genfromtxt('MIT_BIH.csv', delimiter=',')

# Remove NaN values
ecg = ecg[~np.isnan(ecg).any(axis=1)]

# Apply Savitzky-Golay filter to remove noise
ecg = savgol_filter(ecg, window_length=5, polyorder=2, axis=0)

# Apply Kalman filter to remove baseline drift
kf = KalmanFilter(n_dim_obs=ecg.shape[1], n_dim_state=ecg.shape[1])
ecg = kf.em(ecg).smooth(ecg)[0]

# Apply polynomial fitting algorithm to remove baseline wander
for i in range(ecg.shape[1]):
    p = np.polyfit(np.arange(ecg.shape[0]), ecg[:, i], deg=5)
    ecg[:, i] = ecg[:, i] - np.polyval(p, np.arange(ecg.shape[0]))

# Save preprocessed ECG data to CSV file
np.savetxt('preprocessed_ecg_data1.csv', ecg, delimiter=',')

# FEATURE EXTRACTION

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import welch

# Load the preprocessed dataset
preprocessed_data = pd.read_csv('preprocessed_ecg_data1.csv')

# Extract ECG signals from the dataset
ecg_signals = preprocessed_data.iloc[:, 1:].values  # Assuming ECG signals are in columns 1 to 4

# Initialize an empty list to store the extracted features
features = []

# Iterate over each ECG signal in the dataset
for ecg_signal in ecg_signals:
    # Statistical features
    mean = np.mean(ecg_signal)
    std = np.std(ecg_signal)
    kurt = kurtosis(ecg_signal)
    skewness = skew(ecg_signal)
    
    # Frequency-domain features
    f, psd = welch(ecg_signal)
    peak_freq = f[np.argmax(psd)]
    total_power = np.sum(psd)
    
    # Append the features to the list
    features.append([mean, std, kurt, skewness, peak_freq, total_power])

# Convert the list of features to a NumPy array
features = np.array(features)

# Save the extracted features to a new CSV file
feature_df = pd.DataFrame(features, columns=['mean', 'std', 'kurtosis', 'skewness', 'peak_freq', 'total_power'])
feature_df.to_csv('ecg_features9091.csv', index=False)

C:\Users\sakth\OneDrive\Desktop\PYTHON IDE\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 11, using nperseg = 11
  warnings.warn('nperseg = {0:d} is greater than input length '


# SEGMENTATION

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the dataset
dataset = pd.read_csv('ecg_features9091.csv')

# Select the attributes for segmentation
attributes = ['mean', 'std', 'kurtosis', 'skewness', 'peak_freq', 'total_power']
segmentation_data = dataset[attributes]

# Perform Z-score normalization
scaler = StandardScaler()
normalized_data = scaler.fit_transform(segmentation_data)

# Convert the normalized data back to a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=attributes)

# Print the normalized data
print(normalized_df)
normalized_df.to_csv('segment101.csv', index=False)

           mean        std  kurtosis  skewness  peak_freq  total_power
0     16.399851  16.012640 -0.874172 -0.870161  -0.022438    22.705893
1     17.729595  17.464631 -0.667751 -0.909989  -0.022438    24.621334
2     17.541390  17.691539  0.054332 -1.021467  -0.022438    21.925744
3     15.645330  16.228724  0.052383 -1.011028  -0.022438    19.673023
4     15.315998  15.918923 -0.569332 -0.912059  -0.022438    21.793750
...         ...        ...       ...       ...        ...          ...
4054   0.365603   0.099693  0.497942 -1.092687  -0.022438    -0.053639
4055   0.354579   0.091706  0.469110 -1.090322  -0.022438    -0.053721
4056   0.351056   0.091502  0.325874 -1.076074  -0.022438    -0.053406
4057   0.360545   0.099181  0.299558 -1.073489  -0.022438    -0.053190
4058   0.377014   0.110440  0.341809 -1.077281  -0.022438    -0.053011

[4059 rows x 6 columns]


# MODEL BULIDING

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
from tensorflow import keras

# Load the segmented dataset from CSV
dataset = pd.read_csv('segment101.csv')

# Extract features and labels
X = dataset.iloc[:, :5].values  # Adjust the column range according to your dataset
y = dataset.iloc[:, 5].values

# Set the threshold for converting continuous labels to binary
threshold = 0.5

# Convert labels to binary based on the threshold
y_binary = np.where(y >= threshold, 1, 0)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Reshape the data for model input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(5, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = np.where(y_pred >= threshold, 1, 0)
accuracy = (y_test == y_pred_binary).mean()
print("Accuracy:", accuracy)
# Load the model
model.save('cnn_model10100.h5')


Epoch 1/20
102/102 [==============================] - 2s 6ms/step - loss: 0.2571 - accuracy: 0.9794 - val_loss: 0.0405 - val_accuracy: 0.9963
Epoch 2/20
102/102 [==============================] - 0s 3ms/step - loss: 0.0156 - accuracy: 0.9988 - val_loss: 0.0110 - val_accuracy: 0.9988
Epoch 3/20
102/102 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 0.9988
Epoch 4/20
102/102 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 0.9988
Epoch 5/20
102/102 [==============================] - 0s 4ms/step - loss: 9.6537e-04 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 0.9988
Epoch 6/20
102/102 [==============================] - 0s 3ms/step - loss: 6.2767e-04 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 0.9988
Epoch 7/20
102/102 [==============================] - 0s 4ms/step - loss: 4.3665e-04 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 0

# Evaluation metrics

In [16]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Assuming you have loaded the test data and made predictions using the trained model
y_true = y_test  # True labels
y_pred = y_pred_binary  # Predicted labels

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate precision, recall, and F1 score
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = (tp + tn) / (tp + tn + fp + fn)

# Print the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Accuracy:", accuracy)

# Plot ROC curve
import matplotlib.pyplot as plt

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
plt.plot([0, 1], [0, 1], 'k--')  # Add the diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

Precision: 1.0
Recall: 0.6666666666666666
F1 Score: 0.8
Accuracy: 0.9987684729064039


NameError: name 'fpr' is not defined

<Figure size 800x600 with 0 Axes>

# AUTHENTICATION

In [16]:
import pandas as pd
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model("cnn_model10100.h5")

# Load the segmented dataset from CSV
segmented_data = pd.read_csv("one_segment.csv")

# Adjust the shape of the segmented data
segmented_data = segmented_data.values  # Convert to numpy array
segmented_data = np.reshape(segmented_data[:, :5], (segmented_data.shape[0], 5, 1))  # Reshape to (n_samples, 5, 1)

# Feed the reshaped data to the model
prediction = model.predict(segmented_data)

# Make the authentication decision
threshold = 0.5  # Define your threshold
is_authenticated = np.all(prediction > threshold)
.

# Print the authentication decision
if is_authenticated:
    print("Authentication successful")
else:
    print("Authentication failed")


1/1 [==============================] - 0s 194ms/step
Authentication successful


# GUI

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QFileDialog
from PyQt5.QtCore import Qt
import numpy as np
from scipy.signal import savgol_filter
from pykalman import KalmanFilter
import pandas as pd
from PyQt5.QtGui import QPixmap

from scipy.stats import kurtosis, skew
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
from PyQt5.QtGui import QMovie
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QFileDialog
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QVBoxLayout, QWidget, QSizePolicy
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QMovie
from PyQt5.QtWidgets import QMessageBox
class ECGPreprocessingApp(QMainWindow):
    def __init__(self):
        super(ECGPreprocessingApp, self).__init__()
        # Set background GIF
        
        self.setWindowTitle("ECG Preprocessing App")
        self.setGeometry(110, 110, 2000, 1250)
        self.setStyleSheet("QMainWindow::title {font-size: 50pt;}")# Set the font size for the window's title bar

        # Set background GIF
        self.background_label = QLabel(self)
        self.background_label.setGeometry(0, 0, self.width(), self.height())
        self.background_label.setScaledContents(True)  # Scale the image to fit the label
        self.movie = QMovie("70bm.gif")
        self.movie.setScaledSize(self.size())  # Set the size of the GIF to match the window size
        self.background_label.setMovie(self.movie)
        self.movie.start()
            # Set background GIF
        #self.setStyleSheet("background-image: url(ekg-heart-rate.gif); background-repeat: no-repeat; background-position: center;")

          # Set background color
        self.setStyleSheet("background-color: white;")
        # Create labels
        self.status_label = QLabel("ECG AUTHENTICATION", self)
        self.status_label.setGeometry(800, 10, 300, 30)
        self.status_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(15)  # Set the desired font size
        self.status_label.setFont(font)
        
        self.status_label = QLabel("No file selected.", self)
        self.status_label.setGeometry(800, 50, 300, 30)
        self.status_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.status_label.setFont(font)

        self.preprocessing_label = QLabel("Preprocessing: Not completed", self)
        self.preprocessing_label.setGeometry(800, 90, 300, 30)
        self.preprocessing_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.preprocessing_label.setFont(font)
        
        self.feature_extraction_label = QLabel("Feature Extraction: Not completed", self)
        self.feature_extraction_label.setGeometry(800, 130, 300, 30)
        self.feature_extraction_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.feature_extraction_label.setFont(font)
        
        self.segmentation_label = QLabel("Segmentation: Not completed", self)
        self.segmentation_label.setGeometry(800, 170, 300, 30)
        self.segmentation_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.segmentation_label.setFont(font)
        
        self.model_building_label = QLabel("Model Building: Not completed", self)
        self.model_building_label.setGeometry(800, 210, 300, 30)
        self.model_building_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.model_building_label.setFont(font)
        
        self.authentication_label = QLabel("Authentication: Not completed", self)
        self.authentication_label.setGeometry(800, 250, 300, 30)
        self.authentication_label.setAlignment(Qt.AlignCenter)
        font = QFont()
        font.setPointSize(10)  # Set the desired font size
        self.authentication_label.setFont(font)

         # Create buttons
        self.select_file_button = QPushButton("Select File", self)
        self.select_file_button.setGeometry(900, 340, 150, 50)
        self.select_file_button.clicked.connect(self.select_file)
        self.select_file_button.setFont(QFont("Arial", 10))

        self.preprocessing_button = QPushButton("Preprocessing", self)
        self.preprocessing_button.setGeometry(900, 400, 150, 50)
        self.preprocessing_button.setEnabled(False)
        self.preprocessing_button.clicked.connect(self.preprocessing)
        self.preprocessing_button.setFont(QFont("Arial", 10))

        self.feature_extraction_button = QPushButton("Feature Extraction", self)
        self.feature_extraction_button.setGeometry(900, 460, 150, 50)
        self.feature_extraction_button.setEnabled(False)
        self.feature_extraction_button.clicked.connect(self.feature_extraction)
        self.feature_extraction_button.setFont(QFont("Arial", 10))

        self.segmentation_button = QPushButton("Segmentation", self)
        self.segmentation_button.setGeometry(900, 520, 150, 50)
        self.segmentation_button.setEnabled(False)
        self.segmentation_button.clicked.connect(self.segmentation)
        self.segmentation_button.setFont(QFont("Arial", 10))

        self.model_building_button = QPushButton("Model Building", self)
        self.model_building_button.setGeometry(900, 580, 150, 50)
        self.model_building_button.setEnabled(False)
        self.model_building_button.clicked.connect(self.model_building)
        self.model_building_button.setFont(QFont("Arial", 10))

        self.authentication_button = QPushButton("Authentication", self)
        self.authentication_button.setGeometry(900, 640, 150, 50)
        self.authentication_button.setEnabled(False)
        self.authentication_button.clicked.connect(self.authenticate)
        self.authentication_button.setFont(QFont("Arial", 10))
        
        self.is_authenticated = False
        

    def select_file(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Select ECG File", "", "CSV Files (*.csv)")

        if file_path:
            self.status_label.setText("File selected: {}".format(file_path))
            self.file_path = file_path
            self.preprocessing_button.setEnabled(True)

    def preprocessing(self):
        # Load ECG data from CSV file
        ecg = np.genfromtxt(self.file_path, delimiter=',')

        # Remove NaN values
        ecg = ecg[~np.isnan(ecg).any(axis=1)]

        # Apply Savitzky-Golay filter to remove noise
        ecg = savgol_filter(ecg, window_length=5, polyorder=2, axis=0)

        # Apply Kalman filter to remove baseline drift
        kf = KalmanFilter(n_dim_obs=ecg.shape[1], n_dim_state=ecg.shape[1])
        ecg = kf.em(ecg).smooth(ecg)[0]

        # Apply polynomial fitting algorithm to remove baseline wander
        for i in range(ecg.shape[1]):
            p = np.polyfit(np.arange(ecg.shape[0]), ecg[:, i], deg=5)
            ecg[:, i] = ecg[:, i] - np.polyval(p, np.arange(ecg.shape[0]))

        # Save preprocessed ECG data to CSV file
        np.savetxt('preprocessed_ecg_data.csv', ecg, delimiter=',')

        self.preprocessing_label.setText("Preprocessing: Completed")
        self.feature_extraction_button.setEnabled(True)

    def feature_extraction(self):
        # Load the preprocessed dataset
        preprocessed_data = pd.read_csv('preprocessed_ecg_data.csv')

        # Extract ECG signals from the dataset
        ecg_signals = preprocessed_data.iloc[:, 1:].values  # Assuming ECG signals are in columns 1 to 4

        # Initialize an empty list to store the extracted features
        features = []

        # Iterate over each ECG signal in the dataset
        for ecg_signal in ecg_signals:
            # Statistical features
            mean = np.mean(ecg_signal)
            std = np.std(ecg_signal)
            kurt = kurtosis(ecg_signal)
            skewness = skew(ecg_signal)

            # Frequency-domain features
            f, psd = welch(ecg_signal)
            peak_freq = f[np.argmax(psd)]
            total_power = np.sum(psd)

            # Append the features to the list
            features.append([mean, std, kurt, skewness, peak_freq, total_power])

        # Convert the list of features to a NumPy array
        features = np.array(features)

        # Save the extracted features to a new CSV file
        feature_df = pd.DataFrame(features, columns=['mean', 'std', 'kurtosis', 'skewness', 'peak_freq', 'total_power'])
        feature_df.to_csv('ecg_features.csv', index=False)

        self.feature_extraction_label.setText("Feature Extraction: Completed")
        self.segmentation_button.setEnabled(True)

    def segmentation(self):
        # Load the dataset
        dataset = pd.read_csv('ecg_features.csv')

        # Select the attributes for segmentation
        attributes = ['mean', 'std', 'kurtosis', 'skewness', 'peak_freq', 'total_power']
        segmentation_data = dataset[attributes]

        # Perform Z-score normalization
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(segmentation_data)

        # Convert the normalized data back to a DataFrame
        normalized_df = pd.DataFrame(normalized_data, columns=attributes)

        # Save the normalized data to a new CSV file
        normalized_df.to_csv('segmented_data.csv', index=False)

        self.segmentation_label.setText("Segmentation: Completed")
        self.model_building_button.setEnabled(True)

    def model_building(self):
        # Load the segmented dataset from CSV
        segmented_data = pd.read_csv("one_segment.csv")

        # Load the pre-trained model
        model = load_model("cnn_model10100.h5")

        # Adjust the shape of the segmented data
        segmented_data = segmented_data.values  # Convert to numpy array
        segmented_data = np.reshape(segmented_data[:, :5], (segmented_data.shape[0], 5, 1))  # Reshape to (n_samples, 5, 1)

        # Feed the reshaped data to the model
        prediction = model.predict(segmented_data)

        # Make the authentication decision
        threshold = 0.5  # Define your threshold
        self.is_authenticated = np.all(prediction > threshold)

        self.model_building_label.setText("Model Building: Completed")
        self.authentication_button.setEnabled(True)

    def authenticate(self):
        if self.is_authenticated:
            authentication_result = "Authentication successful"
        else:
            authentication_result = "Authentication failed"

        msg_box = QMessageBox()
        msg_box.setText(authentication_result)
        msg_box.exec_()
       # self.authenticate_button.clicked.connect(self.authenticate)
if __name__ == '__main__':
    
    
    
    app = QApplication(sys.argv)
    window = ECGPreprocessingApp()
    window.show()
    sys.exit(app.exec_())

    
    

SystemExit: 0

C:\Users\sakth\OneDrive\Desktop\PYTHON IDE\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
